In [1]:
import pandas as pd
from clickhouse_driver import Client

host = "localhost"
port = 9099
user = "sjjc"
password = "taosan506"
database = "dm"
send_receive_timeout = 10
client = Client(host=host, port=port, user=user, password=password, database=database,
                send_receive_timeout=send_receive_timeout)
credit_df_list_sql = """
select uid, credit_level from dm.pri_credit_info;
"""
credit_df_list = client.execute(credit_df_list_sql)
attr_list = ['uid', 'credit_level']
credit_df = pd.DataFrame(columns=attr_list, data=credit_df_list)
credit_df.set_index('uid', inplace=True)

In [2]:
df= pd.read_csv("./result.csv",encoding='utf-8')
df.set_index('uid', inplace=True)

In [3]:
df.fillna(0,inplace=True)

In [4]:
data_all = pd.merge(credit_df,df,how='left',left_index=True, right_index = True,sort=False,copy=True)

In [5]:
from sklearn.model_selection import train_test_split
data_all['credit_level'] = data_all['credit_level'].apply(int)
known_data = data_all[(data_all['credit_level']!=-1)]
target_data = data_all[(data_all['credit_level']==-1)]
x = known_data.iloc[:,1:]
y = known_data.iloc[:,0:1]['credit_level'].tolist()

In [6]:
x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.25)

In [18]:
for i in [35,50,60,70,85]:
    print(i,"训练集次数",y_train.count(i),"测试集次数",y_test.count(i))

35 训练集次数 6 测试集次数 1
50 训练集次数 93 测试集次数 23
60 训练集次数 14057 测试集次数 4698
70 训练集次数 5984 测试集次数 1972
85 训练集次数 2740 测试集次数 933


In [8]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class='multinomial',solver='newton-cg',max_iter=500)

In [9]:
log_reg.fit(x_train, y_train)
predict_train = log_reg.predict(x_train)

/home/fguohao/.local/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


In [10]:
predict_test = log_reg.predict(x_test)

In [11]:
def true_score(result, test):
    t = 0
    full_def = 0
    for i in range(len(result)):
        if result[i]!=test[i]:
            t = t+1
            full_def = full_def + abs(int(result[i])-int(test[i]))
    print(full_def/len(test))
    return 1-t/len(result)

In [12]:
print(true_score(predict_train, y_train))
print(true_score(predict_test,y_test))

5.328234265734266
0.6251311188811188
5.352694375245838
0.624229710239937


In [13]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, predict_test),
    index = [35,50,60,70,85],
    columns = [35,50,60,70,85]
)
print(cm)

    35  50    60   70  85
35   0   0     1    0   0
50   0   0    20    3   0
60   0   5  4446  241   6
70   0   3  1651  311   7
85   0   0   732  197   4


In [36]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=750
,n_jobs=-1,max_leaf_nodes=10000)
rf_clf.fit(x_train,y_train)


RandomForestClassifier(max_leaf_nodes=10000, n_estimators=750, n_jobs=-1)

In [37]:
y_pred = rf_clf.predict(x_test)

In [38]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [35,50,60,70,85],
    columns = [35,50,60,70,85]
)
print(cm)

    35  50    60    70   85
35   0   0     0     1    0
50   0   0    17     6    0
60   0   0  4162   481   55
70   0   0   642  1168  162
85   0   0   211   537  185


In [39]:
print(true_score(y_test, y_pred))

3.7616362921201
0.7230890258292907


In [50]:
x_target = target_data.iloc[:,1:]
y_target = target_data.iloc[:,0:1]['credit_level'].tolist()
y_result = rf_clf.predict(x_target)
i = 0
ans = []
for item in x_target.iterrows():
    ans.append((item[0],y_result[i]))
    i = i+1

In [51]:
dataans = pd.DataFrame(columns=['uid','credit_level'], data=ans)
dataans.to_csv('credit_level_result.csv',index=False, encoding='utf-8')